In [1]:
# загружаю библиотеки

import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from pymystem3 import Mystem
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [88]:
# загружаю датасет

work = pd.read_excel('C:/Data/Красная Горка/test.xlsx', sheet_name='work')

In [3]:
# смотрю данные

work.sample(3)

,№№,clinic,species,type,type_2,product_original,product,new_text,addition,comment,unit,price_limit,action,price
942,2743,100med,взрослые,3. Терапия,терапия,Установка стандартного металлического абатмент...,Установка стандартного металлического абатмент...,Установка стандартного металлического абатмент...,NaN,NaN,NaN,NaN,NaN,9500
418,1894,PremiumSmile,взрослые,6. Имплантология,Направленная регенерация тканей,операция костной пластики с использованием кос...,операция костной пластики с использованием кос...,операция костной пластики с использованием кос...,NaN,NaN,NaN,NaN,NaN,69300
517,2318,100med,взрослые,2. Профилактика и гигиена,профессиональная гигиена,Обработка одного зуба аппаратом Air Flow,Air Flow,Air Flow,NaN,NaN,1 зуб,NaN,NaN,100


In [4]:
work.dtypes

№№                    int64
clinic               object
species              object
type                 object
type_2               object
product_original     object
product              object
new_text             object
addition             object
comment              object
unit                 object
price_limit          object
action              float64
price                 int64
dtype: object

In [5]:
# меняю ё на е

work.new_text = work.new_text.str.replace('ё', 'е')

In [6]:
# убираю переводы строки

work.new_text = work.new_text.str.replace('\n', ' ')

In [7]:
# удаляю знаки все препинания
work.new_text = work.new_text.str.replace(r'[^\w\s]+', ' ')

<ipython-input-7-63082e6ea99b>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  work.new_text = work.new_text.str.replace(r'[^\w\s]+', ' ')


In [8]:
# убираю цифры

#work.new_text = work.new_text.str.replace(r'[0-9]+', ' ')

In [9]:
# убираю двойные пробелы

work.new_text = work.new_text.str.replace('  ', ' ')

In [27]:
# перевожу в нижний регистр

work.new_text = work.new_text.str.lower()

In [29]:
work.sample(2)

,№№,clinic,species,type,type_2,product_original,product,new_text,addition,comment,unit,price_limit,action,price
684,2485,100med,взрослые,4. Хирургия,удаление зубов,Удаление зуба подвижного,Удаление подвижного зуба,удаление зуба подвижного,NaN,NaN,NaN,NaN,NaN,1400
964,5895,KRH Dental & Medical,взрослые,1. Общее,анестезия,инфильтрационная,Анестезия инфильтрационная,анестезия инфильтрационная,NaN,NaN,1,NaN,NaN,500


In [30]:
# сортирую слова по алфавиту

work['new_text'] = work['new_text'].apply(lambda x: ' '.join(sorted(x.split(' '))))

In [31]:
work.head(10)

,№№,clinic,species,type,type_2,product_original,product,new_text,addition,comment,unit,price_limit,action,price
0,1,Doctor Art,взрослые,3. Терапия,консультация,Первичная консультация врача-стоматолога (осмо...,Консультация стоматолога первичная,консультация первичная стоматолога,NaN,NaN,1,NaN,NaN,0
1,2,Doctor Art,взрослые,1. Общее,анестезия,Анестезия инфильтрационная,Анестезия инфильтрационная,анестезия инфильтрационная,NaN,NaN,1,NaN,NaN,450
2,3,Doctor Art,взрослые,1. Общее,анестезия,Анестезия проводниковая,Анестезия проводниковая,анестезия проводниковая,NaN,NaN,1,NaN,NaN,550
3,4,Doctor Art,взрослые,2. Профилактика и гигиена,профессиональная гигиена,"Комплексная проф.гигиена всей полости рта (УЗ,...","Профгигиена (УЗ, Air Flow, полировка)",air flow полировка профгигиена уз,NaN,NaN,2 челюсти,NaN,NaN,3500
4,5,Doctor Art,взрослые,2. Профилактика и гигиена,профессиональная гигиена,"Покрытие зубов фторсодержащими препаратами, 1 зуб",Фторирование эмали,фторирование эмали,NaN,NaN,1 зуб,NaN,NaN,100
5,6,Doctor Art,взрослые,2. Профилактика и гигиена,профессиональная гигиена,"Устранение гиперчувствительности, 1 зуб",Лечение гиперчувствительности,гиперчувствительности лечение,NaN,NaN,1 зуб,NaN,NaN,300
6,7,Doctor Art,взрослые,2. Профилактика и гигиена,отбеливание зубов,"Профессиональное отбеливание системой ZOOM 3, ...",Отбеливание Zoom-3,3 zoom отбеливание,NaN,NaN,1 челюсть,NaN,NaN,5500
7,8,Doctor Art,взрослые,3. Терапия,лечение кариеса,"Лечение кариеса (без учета анестезии), min","Лечение кариеса, без стоимости анестезии",анестезии без кариеса лечение стоимости,NaN,NaN,1 зуб,min,NaN,2100
8,9,Doctor Art,взрослые,3. Терапия,лечение пульпита,"Лечение пульпита, min","Лечение пульпита, без стоимости анестезии",анестезии без лечение пульпита стоимости,NaN,"в прайсе ""без стоимости"" пропущено",1 зуб,min,NaN,2300
9,10,Doctor Art,взрослые,4. Хирургия,удаление зубов,"Удаление зуба (без учета анестезии), min","Удаление зуба, без стоимости анестезии",анестезии без зуба стоимости удаление,NaN,NaN,1 зуб,min,NaN,850


In [38]:
need_text = str(work.new_text)

In [34]:
# !pip uninstall spacy
# !pip install -U spacy
# !python -m spacy download ru_core_news_sm

import spacy ## общий пакет, где лежит весь функционал
import ru_core_news_sm ## натренированные модели на русском
import string ## здесь возомьм пунктуацию
import nltk # удобный пакет для всякой работы с текстом

In [39]:
nlp = ru_core_news_sm.load() ## умная "обертка" для каждого текста
doc = nlp(need_text)
doc

0                консультация первичная стоматолога
1                        анестезия инфильтрационная
2                           анестезия проводниковая
3                 air flow полировка профгигиена уз
4                                фторирование эмали
                           ...                     
1133        inno имплант на установка формирователя
1134    имплант на установка формирователя штраумен
1135    anyridge имплант на установка формирователя
1136                имплант керамическая коронка на
1137          zro2 безметалловая имплант коронка на
Name: new_text, Length: 1138, dtype: object

In [40]:
tokenized1 = [token for token in doc] # первый способ

In [ ]:
tokenized2 = text.split(' ') # второй способ

In [41]:
from spacy.lang.ru.stop_words import STOP_WORDS # стоп слова


[token for token in doc if token not in STOP_WORDS]

[0,
                ,
 консультация,
 первичная,
 стоматолога,
 ,
 1,
                        ,
 анестезия,
 инфильтрационная,
 ,
 2,
                           ,
 анестезия,
 проводниковая,
 ,
 3,
                 ,
 air,
 flow,
 полировка,
 профгигиена,
 уз,
 ,
 4,
                                ,
 фторирование,
 эмали,
 
                            ,
 ...,
                     ,
 1133,
        ,
 inno,
 имплант,
 на,
 установка,
 формирователя,
 ,
 1134,
    ,
 имплант,
 на,
 установка,
 формирователя,
 штраумен,
 ,
 1135,
    ,
 anyridge,
 имплант,
 на,
 установка,
 формирователя,
 ,
 1136,
                ,
 имплант,
 керамическая,
 коронка,
 на,
 ,
 1137,
          ,
 zro2,
 безметалловая,
 имплант,
 коронка,
 на,
 ,
 Name,
 :,
 new_text,
 ,,
 Length,
 :,
 1138,
 ,,
 dtype,
 :,
 object]

In [42]:
#" ".join(stemmer.stem(token) for token in tokens)
lemmatizer = nltk.stem.WordNetLemmatizer()
#" ".join(lemmatizer.lemmatize(token) for token in tokens)

In [46]:
punctuations = string.punctuation
stopwords = STOP_WORDS

def spacy_processor(sentence):
    mytokens = nlp(sentence)
    mytokens = [ word.lemma_.lower().strip() for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens
spacy_processor(need_text)

'0 консультация первичный стоматолог 1 анестезия инфильтрационный 2 анестезия проводниковый 3 air flow полировка профгигиена уз 4 фторирование эмаль ... 1133 inno имплант установка формирователь 1134 имплант установка формирователь штраумен 1135 anyridge имплант установка формирователь 1136 имплант керамический коронка 1137 zro2 безметалловая имплант коронка name new_text length 1138 dtype object'

In [47]:
work['new_text'] = work['new_text'].apply(spacy_processor)

In [48]:
work

,№№,clinic,species,type,type_2,product_original,product,new_text,addition,comment,unit,price_limit,action,price
0,1,Doctor Art,взрослые,3. Терапия,консультация,Первичная консультация врача-стоматолога (осмо...,Консультация стоматолога первичная,консультация первичный стоматолог,NaN,NaN,1,NaN,NaN,0
1,2,Doctor Art,взрослые,1. Общее,анестезия,Анестезия инфильтрационная,Анестезия инфильтрационная,анестезия инфильтрационный,NaN,NaN,1,NaN,NaN,450
2,3,Doctor Art,взрослые,1. Общее,анестезия,Анестезия проводниковая,Анестезия проводниковая,анестезия проводниковая,NaN,NaN,1,NaN,NaN,550
3,4,Doctor Art,взрослые,2. Профилактика и гигиена,профессиональная гигиена,"Комплексная проф.гигиена всей полости рта (УЗ,...","Профгигиена (УЗ, Air Flow, полировка)",air flow полировка профгигиена уз,NaN,NaN,2 челюсти,NaN,NaN,3500
4,5,Doctor Art,взрослые,2. Профилактика и гигиена,профессиональная гигиена,"Покрытие зубов фторсодержащими препаратами, 1 зуб",Фторирование эмали,фторирование эмаль,NaN,NaN,1 зуб,NaN,NaN,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133,6064,KRH Dental & Medical,взрослые,6. Имплантология,NaN,Установка формирователя на имплант INNO,Установка формирователя на имплант INNO,inno имплант установка формирователь,NaN,NaN,NaN,NaN,NaN,5000
1134,6065,KRH Dental & Medical,взрослые,6. Имплантология,NaN,Установка формирователя на имплант Штраумен,Установка формирователя на имплант Штраумен,имплант установка формирователь штраумен,NaN,NaN,NaN,NaN,NaN,10000
1135,6066,KRH Dental & Medical,взрослые,6. Имплантология,NaN,Установка формирователя на имплант AnyRidge,Установка формирователя на имплант AnyRidge,anyridge имплант установка формирователь,NaN,NaN,NaN,NaN,NaN,10000
1136,6067,KRH Dental & Medical,взрослые,6. Имплантология,NaN,Коронка керамическая на имплант,Коронка керамическая на имплант,имплант керамический коронка,NaN,NaN,NaN,NaN,NaN,20000


In [49]:
work.to_excel('C:/Data/Красная Горка/temp.xlsx')